# RADIA Example 6: Simple Quadrupole Magnet

This _Jupyter_ notebook devloped on the basis of the original
[Radia Example&#160;6](
  https://www.esrf.fr/Accelerators/Groups/InsertionDevices/Software/Radia/Documentation,
  "RADIA Example 6 at ESRF").

### _Dan T. Abell, RadiaSoft LLC_

$\rule[2pt]{15mm}{0.33pt}\ \LaTeX\ \text{macros}\ \rule[2pt]{15mm}{0.33pt}$
$$
%% math text
\newcommand{\mhsp}{\mskip{1.5mu}}
\newcommand{\hmhsp}{\mskip{0.75mu}}
\newcommand{\nmhsp}{\mskip{-1.5mu}}
\newcommand{\nhmhsp}{\mskip{-0.75mu}}
\newcommand{\ud}{\mathop{}\!\mathrm{d}}% upright d for differential
\newcommand{\ue}{\mathrm{e}}% upright e for Euler number
\newcommand{\ui}{\mathrm{i}}% upright i for unit imaginary
\newcommand{\uj}{\mathrm{j}}% upright j for unit imaginary
\newcommand{\uk}{\mathrm{k}}% upright k for unit imaginary
\newcommand{\sl}{\,/\,}
%%
%% derivatives
\newcommand{\dd}[3][]{\ud^{#1}{#2}/\nmhsp\ud{#3}^{#1}}
\newcommand{\dt}[2][]{\ud^{#1}{#2}/\nmhsp\ud{t}^{#1}}
\newcommand{\Dd}[3][]{\frac{\ud^{#1}{#2}}{\ud{#3}^{#1}}}
\newcommand{\Dt}[2][]{\frac{\ud^{#1}{#2}}{\ud{t}^{#1}}}
\newcommand{\ptdd}[3][]{\partial^{#1}{#2}/\partial{#3}^{#1}}
\newcommand{\ptDd}[3][]{\frac{\partial^{#1}{#2}}{\partial{#3}^{#1}}}
%%
%% vector operators
\DeclareMathOperator{\grad}{\nabla\nmhsp\nmhsp}
\DeclareMathOperator{\divrg}{{\nabla\cdot}\nmhsp\nhmhsp}
\DeclareMathOperator{\curl}{{\nabla\times}\nmhsp\nhmhsp}
%%
%% vectors
%% -- using \boldsymbol
% \newcommand{\uV}[1]{\hat{\boldsymbol{#1}}}% unit vector
% \newcommand{\V}[1]{\boldsymbol{#1}}% vector
% \newcommand{\uVg}[1]{\hat{\boldsymbol{#1}}}% unit vector
% \newcommand{\Vg}[1]{\boldsymbol{#1}}% vector
%% -- using \vec
\newcommand{\uV}[1]{\hat{{#1}}}% unit vector
\newcommand{\V}[1]{\vec{#1}}% vector
\newcommand{\uVg}[1]{\hat{{#1}}}% unit vector
\newcommand{\Vg}[1]{\vec{#1}}% vector
$$
$\rule[2pt]{58.3mm}{0.33pt}$

---
## Introduction & Theory

In this example, we model a simple iron-dominated quadrupole magnet.
The pole-tips have hyperbolic faces with a chamfer at the ends.

Field computations in the case of iron-dominated geometries present specific
difficulties that usually make them less accurate than those in the case of
structures dominated by coils or permanent magnets. Nevertheless, Radia
includes special methods that enable one to obtain reasonable precision
with a reasonable amount of computational effort—cpu time and memory usage.

Because this example bears some similarities to that of Example&#160;5,
all the remarks made there also apply here. We recommend that you at least
review the introduction of that example before playing with this one.
But, as a brief reminder for those in a hurry, the following recommendations
will help you achieve an acceptable level of precision within a reasonable
time frame:

* Segment the corners of iron circuits as parallel or as perpendicular
as possible to lines of magnetic flux. For right-angled corners, one can
do this using the circular or ellipsoidal mode of segmentation (see below). 
Following this recommendation will have a significant impact on your
simulations of iron-dominated electromagnets.

    In the example shown here, we make use of circular segmentation twice
(with the other two corners addressed by symmetry). See the function
`create_quadrupole(..)` in the section below
entitled **_Define a general function to build a quadrupole magnet_**,
and look for the lines containing

```
rad.ObjDivMag(g3, [nr3, np3, nx], 'cyl', cy)
```

    and
    
```
rad.ObjDivMag(g5, [nr5, np5, nx], 'cyl', cy)
```

* Use a finer segmentation for the iron regions (particularly the pole
pieces) closest to the region of interest. 

* Start with a coarse segmentation and gradually make it finer until
the computed field values are stable.
Be aware that both memory usage and cpu time tend to increase as the
square of the number of elements (segments) in the iron.

* To the greatest extent possible, take advantage of any symmetries
present in your system. Doing so saves both memory usage and CPU time.

For an explanation of all Radia functions, see the
[Radia Reference Guide](
  https://www.esrf.eu/Accelerators/Groups/InsertionDevices/Software/Radia/Documentation/ReferenceGuide/Index
  "RADIA Reference Guide at ESRF").

---
## Preamble

### Required imports

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import scipy.constants as sc
import numpy # as np
import time as tm
from math import *
import os

In [ ]:
import radia as rad
from uti_plot import *
import ipywidgets
from jupyter_rs_radia import radia_viewer

The following figure illustrates the simple quadrupole magnet
we simulate in this example.

In [ ]:
# import an illustration of this magnet
from IPython.display import Image
# Image(filename=('./IronQuadrupole.png'))

### Notebook directory

Where are we?

In [ ]:
nb_dir = os.getcwd() + '/'
nb_dir

### Mathematical and physical constants

Define some mathematical constants:

In [ ]:
# pi
# π = np.pi
π = pi

# golden ratio
# gr = (1 + np.sqrt(5)) / 2
gr = (1 + sqrt(5)) / 2

# roots
# rt2 = np.sqrt(2.)
rt2 = sqrt(2.)

# degree to radian, and radian to degree
degree = π / 180.
d2r = degree
r2d = 1 / degree

### Function definitions (utilities)

In [ ]:
# harmonic analysis

def harm(obj, y, z, ro, np):
    '''
    perform ...
    arguments:
        obj = 
        y   = 
        z   = 
        ro  = 
        np  = 
    return list [, , ]
    '''
    arHarm = [complex(0,0)] * np
    d_tet = 2*pi/np
    tet = 0
    for i in range(np):
        cosTet = cos(tet); sinTet = sin(tet)
        re = rad.FldInt(obj, 'inf', 'ibz', [-1,y+ro*cosTet,z+ro*sinTet], [1,y+ro*cosTet,z+ro*sinTet])
        im = rad.FldInt(obj, 'inf', 'iby', [-1,y+ro*cosTet,z+ro*sinTet], [1,y+ro*cosTet,z+ro*sinTet])
        arHarm[i] = complex(re, im)
        tet += d_tet
    return [np, ro, arHarm]

def multipole(w, q):
    np = w[0]; ro = w[1]; arHarm = w[2]
    s = 0
    for p in range(np):
        arg = -2*pi/np*q*p
        s += arHarm[p]*complex(cos(arg), sin(arg))
    return s/np/(ro**q)    

## _Define a general function to build a quadrupole focusing magnet_

Here we define a function that creates the quadrupole focusing magnet
of this example. The various arguments (detailed in the function’s
docstring) specify the geometry, material properties, current, and 
segmentation of this model magnet.

In [ ]:
def create_quadrupole():
    '''
    create a simple quadrupole focusing electromagnet
    arguments:
        gap     = distance between ... / mm
        thick   = thickness of iron pole tip (along particle trajectory) / mm
        width   = width of ... / mm
        height  = height of ... / mm
        chamfer = size of chamfer on ends of iron pole tip / mm
        np
        Nn
        Rmin
        ...
    return Radia representation of a simple quadrupole focusing magnet
    '''

    rap = 0.5
    ct = [0, 0, 0]
    z0 = gap / 2
    y0 = width / 2
    amax = hyp * asinh(y0 / z0)
    dz = z0 * (cosh(amax) - 1)
    aStep = amax / np
    na = int(amax * (1 + 2 / np) / aStep) + 1
    qq = [[z0 * sinh(ia * aStep / hyp), z0 * cosh(ia * aStep)] for ia in range(na)]
    hh = qq[np][1] + height * rap - dz
    qq[np + 1] = [qq[np][0], hh]
    qq[np + 2] = [0, hh]
    g1 = rad.ObjThckPgn(thick / 4, thick / 2, qq)   
    rad.ObjDivMag(g1, n1)

    # vertical segment on top of pole faces
    g2 = rad.ObjRecMag([thick / 4, width / 4, gap / 2 + height * (1 / 2 + rap / 2)],
                       [thick / 2, width / 2, height * (1 - rap)])
    rad.ObjDivMag(g2, n2)

    # corner
    gg = rad.ObjCnt([g1, g2])
    gp = rad.ObjCutMag(gg, [thick / 2 - chamfer - gap / 2, 0, 0], [1, 0, -1])[0]
    g3 = rad.ObjRecMag([thick / 4, width / 4, gap / 2 + height + depth / 2],
                       [thick / 2, width / 2, depth])
    cy = [[[0, width / 2, gap / 2 + height], [1, 0, 0]], [0, 0, gap / 2 + height], 2 * depth / width]
    rad.ObjDivMag(g3, [nr3, np3, nx], 'cyl', cy)

    # horizontal segment between the corners
    tan_n = tan(2 * pi / 2 / Nn)
    length = tan_n * (height + gap / 2) - width / 2
    g4 = rad.ObjRecMag([thick / 4, width / 2 + length / 2, gap / 2 + height + depth / 2],
                       [thick / 2,length, depth])
    rad.ObjDivMag(g4, n4)

    # other corner
    posy = width / 2 + length
    posz = posy / tan_n
    g5 = rad.ObjThckPgn(thick / 4, thick / 2, [[posy, posz],[posy, posz + depth], [posy + depth * tan_n, posz + depth]])
    cy = [[[0, posy, posz], [1, 0, 0]], [0, posy, posz + depth], 1]
    rad.ObjDivMag(g5, [nr5, np5, nx], 'cyl', cy)

    # generation of the coil
    Rmax = Rmin - width/2 + gap/2 + offset - 2
    coil1 = rad.ObjRaceTrk([0,0,gap/2+height/2+offset/2], [Rmin,Rmax], [thick,width-2*Rmin], height-offset, 3, CurDens)
    rad.ObjDrwAtr(coil1, coilcolor)
    hh = (height - offset)/2
    coil2 = rad.ObjRaceTrk([0,0,gap/2+height-hh/2], [Rmax,Rmax+hh*0.8], [thick,width-2*Rmin], hh, 3, CurDens)
    rad.ObjDrwAtr(coil2, coilcolor)

    # make container, set colors, and define symmetries
    g = rad.ObjCnt([gp, g3, g4, g5])
    rad.ObjDrwAtr(g, ironcolor)
    gd = rad.ObjCnt([g])
    
    rad.TrfZerPerp(gd, ct, [1, 0, 0])
    rad.TrfZerPerp(gd, ct, [0, 1, 0])
    
    t = rad.ObjCnt([gd, coil1, coil2])
    rad.TrfZerPara(t, ct, [0, cos(pi / Nn), sin(pi / Nn)])

    rad.TrfMlt(t, rad.TrfRot(ct, [1, 0, 0], 4 * pi / Nn), int(round(Nn / 2)))
    rad.MatApl(g, ironmat)
    rad.TrfOrnt(t, rad.TrfRot([0, 0, 0],[1, 0, 0],pi / Nn))
    
    return t

## _Create the quadrupole and solve for the fields_

First set the various parameters that specify the properties—geometry,
materials, and current—of our quadrupole. Then also decide how finely
to segment the iron.

In [ ]:
# general parameters for quadrupole geometry
np = 4
hyp = 1      # must be >0.2 !
gap = 40     # magnetic gap / mm
width = 30   # pole width / mm
height = 50  # height of iron yoke / mm
thick = 60   # length of the quad / mm
chamfer = 8  # size of chamfer

Nn = 4
ironcolor = [0.0, 1.0, 1.0]
coilcolor = [1.0, 0.0, 0.0]
depth = 1.2 * width / 2

#Coils
CurDens = -3 # current density / (A / mm^2)
Rmin = 2
offset = 10

#Segmentation Params
nx = 2
ny = 2
n1 = [nx,ny,3]
n2 = [nx,ny,3]
np3 = 2
nr3 = ny
n4 = [nx,3,ny]
np5 = ceil(np3/2); print()
nr5 = ny

Now create and display this quadrupole magnet:

In [ ]:
rad.UtiDelAll()
ironmat = rad.MatSatIsoFrm([2000,2],[0.1,2],[0.1,2])
t0 = tm.time()
quad = create_quadrupole()
t1 = tm.time()
size = rad.ObjDegFre(quad)

print('Built in time', round((t1 - t0) * 1e3, 3),'ms')
print('Interaction matrix:', size, 'x', size, '.equiv.', (4 * size * size / 1e6), 'MBytes')

# set up the radia viewer and display the magnet
rv = radia_viewer.RadiaViewer()
rv.add_geometry('Quadrupole Magnet', quad)
rv.display()

In [ ]:
# solve for the magnetization
prec     = 10.e-6 # precision for this computation
max_iter = 10000  # maximum allowed iterations
t0  = tm.time()
res = rad.Solve(quad, prec, max_iter)
t1  = tm.time()

print("Solved for magnetization in time {0:6f} s".format(t1 - t0))
print("Relaxation results")
print("  number of iterations: {0:5d}".format(int(res[3])))
if(res[3] == max_iter):
    print("    >> unstable or incomplete relaxation")
print("  average stability of magnetization at last iteration: {0:.4e} T".format(res[0]))
print("  maximum absolute magnetization at last iteration: {0:.5f} T".format(res[1]))
print("  maximum H vector at last iteration: {0:.5f} T".format(res[2]))
# print("Pole-tip magnetic field: {0:.8f} T".format(rad.Fld(quad, 'bz', [x,y,z])))b

The quadrupole gradient ... $B_z$ field at the origin, $(0, 0, 0)$, is the vertical
field in the middle of the gap.

In [ ]:
Bz  = rad.Fld(quad, 'Bz', [0, 1, 0]) * 1e3
Bz1 = rad.Fld(quad, 'Bz', [0,10, 0]) * 1e3 / 10
Iz  = rad.FldInt(quad, 'inf', 'ibz', [-1, 1, 0], [1, 1, 0])
Iz1 = rad.FldInt(quad, 'inf', 'ibz', [-1,10, 0], [1,10, 0]) / 10

print(' quadrupole gradient:', round(Bz, 4), 'T/m')
print('  int.quad. at  1 mm:', round(Iz, 5), 'T')
print('δ int.quad. at 10 mm:', round((Iz1 / Iz - 1), 6) * 100, '%') # rel. var. in field integral

## _Plots of the magnetic field and field integrals_

Here we show plots of magnetic field in the gap and corresponding
field integrals. The field values are obtained by calling `Fld` on
a list of points. One may also use `FldLst`.

The first graph here displays ...

In [ ]:
# plots of magnetic field
z    = 0
x1   = 0
x2   = 30
ymax = 40
np   = 20

BzVy1 = rad.FldLst(quad, 'bz', [x1, 0, z], [x1, ymax, z], np, 'arg', 0)
BzVy2 = rad.FldLst(quad, 'bz', [x2, 0, z], [x2, ymax, z], np, 'arg', 0)

uti_plot1d_m([BzVy1, BzVy2],
             labels = ['Y', 'Vertical Magnetic Field', 'Vertical Magnetic Field vs. Horizontal Position'], units = ['mm', 'T'],
             styles = ['-b.', '-r.'], legend = ['X = {} mm'.format(x1), 'X = {} mm'.format(x2)])

xmax = 1.5 * thick
y1 = width / 4
y2 = width / 2

BzVx1 = rad.FldLst(quad, 'bz', [-xmax, y1, 0], [xmax, y1, 0], 2 * np, 'arg', 0)
BzVx2 = rad.FldLst(quad, 'bz', [-xmax, y2, 0], [xmax, y2, 0], 2 * np, 'arg', 0)

uti_plot1d_m([BzVx1, BzVx2],
             labels = ['X', 'Vertical Magnetic Field', 'Vertical Magnetic Field vs. Longitudinal Position'], units = ['mm', 'T'],
             styles = ['-b.', '-r.'], legend = ['Y = {} mm'.format(y1), 'Y = {} mm'.format(y2)])

# plots of magnetic field integrals
z    = 0
ymin = 0.001
ymax = 10
npy  = 20
yStep = (ymax - ymin) / (npy - 1)
IBz0 = rad.FldInt(quad, 'inf', 'ibz', [-1, 1, z], [1, 1, z])

IBzVsY = [(rad.FldInt(quad, 'inf', 'ibz', [-1, ymin + iy * yStep, z], [1,ymin+iy*yStep,z]) /
           ((ymin+iy*yStep)*IBz0) - 1) * 100 for iy in range(npy)]
uti_plot1d(IBzVsY, [ymin, ymax, npy],
           ['Y', 'dIz', 'Rel. Variation of Vertical Field Integral along X at Z = ' + repr(z) + ' mm'], ['mm', '%'])

# harmonic analysis of the field integrals
nharm = 10; radius = 2; y = 0; z = 0;
w = harm(quad, y, z, radius, nharm);

mm = [multipole(w, i) for i in range(nharm)];
round_mm = [complex(round(mm[i].real, 9), round(mm[i].imag, 9)) for i in range(nharm)];
print(round_mm)

uti_plot_show()

Save the above data for testing:

In [ ]:
numpy.savetxt(nb_dir + "BzVy1.txt", BzVy1)
numpy.savetxt(nb_dir + "BzVy2.txt", BzVy2)
numpy.savetxt(nb_dir + "BzVx1.txt", BzVx1)
numpy.savetxt(nb_dir + "BzVx2.txt", BzVx2)

## _Define a general function to build a multipole magnet_

Here we define a function that creates the multipole magnet. The various
arguments (detailed in the function’s docstring) specify the geometry,
material properties, current, and segmentation of this model magnet.

In [ ]:
def create_multipole(n_poles, thick, width, gap, height, chamfer, tip_coil_sep, curr_density,
                     n_curve = 6, r_min = 2., clearance = 2., poletip_frac = 0.5, yoketip_frac = 0.6):
    '''
    Return a Radia representation of a simple multipole electromagnet.

    Arguments:
        npoles       = number of magnet pole tips (even integer)
        thick        = length of iron pole tip (along particle trajectory) / mm
        width        = width of pole tip / mm
        gap          = distance between opposing pole tips / mm
        height       = height of pole tip  / mm
        chamfer      = size of chamfer on pole tip ends / mm
        tip_coil_sep = distance by which coil edge is set back from the pole tip / mm
        curr_density = current density / (A / mm^2)
        n_curve      = number of intervals for discretizing (half) the pole face
        r_min        = minimum coil radius / mm
        clearance    = clearance between coil corner and diagonal between sectors / mm
        poletip_frac = lower fraction of pole tip, possibly subject to finer segmentation
        yoketip_frac = ratio of yoke height (or depth) to pole tip width

    This function constructs one half of a vertical sector of a multipole magnet.
    It then applies appropriate symmetries to construct and orient the full magnet.
    '''
    # sanity checks on user input
    # :: even number of magnetic poles?
    assert n_poles % 2 == 0, "Argument n_poles must equal an even integer."
    # :: pole tips do not intersect? ==> automatic!! <==
    # :: lower pole tip includes all of pole face?
    # :: coil_1 does not cross sector diagonal
    # :: coil_2 does not cross sector diagonal

    # define a few useful vectors
    ctr = [0, 0, 0]
    x_hat = [1, 0, 0]
    y_hat = [0, 1, 0]
    z_hat = [0, 0, 1]

    # discretize hyperbola that defines the pole tip
    # :: z^2 - (hyp * y)^2 = z0^2, w/ asymptotes z = ±hyp * y
    tan_np = tan(pi / n_poles)
    hyp = 1 / tan_np  # slope of hyperbola's asymptote
    z0 = gap / 2
    ym = width / 2
    a_max =  asinh(hyp * ym / z0)
    da = a_max / n_curve
    na = n_curve + 3  # go two points beyond so we don't have to extend the array
    qq = [ [(z0 / hyp) * sinh(ia * da), z0 * cosh(ia * da)] for ia in range(na) ]
    # and
    # modify last two points so as to outline lower portion of the (half) pole tip
    ht_lower = poletip_frac * height
    qq[n_curve + 1] = [ym, z0 + ht_lower]
    qq[n_curve + 2] = [ 0, z0 + ht_lower]

    # create and segment the lower portion of the (half) pole tip
    g1 = rad.ObjThckPgn(thick / 4, thick / 2, qq)
    rad.ObjDivMag(g1, n1)

    # create and segment the upper portion of the (half) pole tip
    ht_upper = height - ht_lower
    g2 = rad.ObjRecMag([thick / 4, width / 4, z0 + height - ht_upper / 2],
                       [thick / 2, width / 2, ht_upper])
    rad.ObjDivMag(g2, n2)

    # combine the lower and upper portions of the (half) pole tip
    gg = rad.ObjCnt([g1, g2])
    # and
    # cut chamfer, then retain desired metal
    gp = rad.ObjCutMag(gg, [thick / 2 - chamfer, 0, z0], [1, 0, -1])[0]

    # create and segment "corner" above (half) pole tip
    depth = yoketip_frac * width
    g3 = rad.ObjRecMag([thick / 4, width / 4, z0 + height + depth / 2],
                       [thick / 2, width / 2, depth])
    cy = [[[0, width / 2, gap / 2 + height], [1, 0, 0]], [0, 0, gap / 2 + height], 2 * depth / width]
    rad.ObjDivMag(g3, [nr3, np3, nx], 'cyl', cy)

    # create and segment horizontal yoke segment to corner
    length = tan_np * (gap / 2 + height) - width / 2
    g4 = rad.ObjRecMag([thick / 4, width / 2 + length / 2, gap / 2 + height + depth / 2],
                       [thick / 2,length, depth])
    rad.ObjDivMag(g4, n4)

    # outline the corner
    yc = width / 2 + length
    zc = gap / 2 + height
    q_corner = [[yc, zc], [yc, zc + depth], [yc + depth * tan_np, zc + depth]]
    # and
    # create and segment yoke corner
    g5 = rad.ObjThckPgn(thick / 4, thick / 2, q_corner)
    cy = [[[0, yc, zc], [1, 0, 0]], [0, yc, zc + depth], 1]
    rad.ObjDivMag(g5, [nr5, np5, nx], 'cyl', cy)

    # create container for all the metal
    g = rad.ObjCnt([gp, g3, g4, g5])
    gd = rad.ObjCnt([g])
    # and set color for iron
    rad.ObjDrwAtr(g, ironcolor)

    # create coil1
    ht_coil = height - tip_coil_sep
    wd_to_diagonal = (gap / 2 + tip_coil_sep) * tan_np
    r_max = wd_to_diagonal - clearance - width / 2 + r_min
    coil1 = rad.ObjRaceTrk([0, 0, z0 + height - ht_coil / 2], [r_min, r_max],
                           [thick, width - 2 * r_min],
                           ht_coil, 3, curr_density)

    # create coil2
    ht_coil = (height - tip_coil_sep) / 2
    wd_to_diagonal = (gap / 2 + height - ht_coil) * tan_np
    # r2 = wd_to_diagonal - clearance - width / 2 + r_min
    r2 = r_max + 0.8 * ht_coil
    coil2 = rad.ObjRaceTrk([0, 0, z0 + height - ht_coil / 2], [r_max, r2],
                           [thick, width - 2 * r_min],
                           ht_coil, 3, curr_density)

    # set color for coils
    rad.ObjDrwAtr(coil1, coilcolor)
    rad.ObjDrwAtr(coil2, coilcolor)

    # apply symmetries to create full pole tip plus attached yoke
    ctr = [0, 0, 0]
    x_hat = [1, 0, 0]
    # reflection in y-z plane, with zero field perpendicular to the plane
    rad.TrfZerPerp(gd, ctr, x_hat)
    # reflection in z-x plane, with zero field perpendicular to the plane
    rad.TrfZerPerp(gd, ctr, y_hat)

    # create container for full pole tip with attached yoke and coils
    t = rad.ObjCnt([gd, coil1, coil2])

    # reflection across diagonal plane, with zero field parallel to the plane
    rad.TrfZerPara(t, ctr, [0, cos(pi / n_poles), sin(pi / n_poles)])
    # ==>> at this point we have a matched pair of pole tips
    #      they subtend an angle 2 * (2π / n_poles) = 4π / n_poles

    # apply symmetries to create full multipole electromagnet
    rad.TrfMlt(t, rad.TrfRot(ctr, x_hat, 4 * pi / n_poles), int(n_poles / 2))
    rad.MatApl(g, ironmat)
    rad.TrfOrnt(t, rad.TrfRot(ctr, x_hat, pi / n_poles))

    return t

In [ ]:
# general parameters for the quadrupole
n_poles =  4   # number of pole tips
thick   = 60.  # length of magnet / mm
width   = 30.  # pole width / mm
gap     = 40.  # magnetic gap / mm
height  = 50.  # height of pole tip / mm
chamfer =  8.  # size of chamfer
tip_coil_sep = 10.
curr_density = -3.  # current density / (A / mm^2)

# segmentation parameters
nx = 2
ny = 2
n1 = [nx,ny,3]
n2 = [nx,ny,3]
np3 = 2
nr3 = ny
n4 = [nx,3,ny]
np5 = ceil(np3/2)
nr5 = ny

# colors
ironcolor = [0.0, 1.0, 1.0]
coilcolor = [1.0, 0.0, 0.0]

In [ ]:
rad.UtiDelAll()
ironmat = rad.MatSatIsoFrm([2000,2],[0.1,2],[0.1,2])
t0 = tm.time()
quad = create_multipole(n_poles, thick, width, gap, height, chamfer, tip_coil_sep, curr_density)
t1 = tm.time()
size = rad.ObjDegFre(quad)

print('Built in time', round((t1 - t0) * 1e3, 3),'ms')
print('Interaction matrix:', size, 'x', size, '.equiv.', (4 * size * size / 1e6), 'MBytes')

# set up the radia viewer and display the magnet
rv = radia_viewer.RadiaViewer()
rv.add_geometry('Quadrupole Magnet', quad)
rv.display()

In [ ]:
# solve for the magnetization
prec     = 10.e-6 # precision for this computation
max_iter = 10000  # maximum allowed iterations
t0  = tm.time()
res = rad.Solve(quad, prec, max_iter)
t1  = tm.time()

print("Solved for magnetization in time {0:6f} s".format(t1 - t0))
print("Relaxation results")
print("  number of iterations: {0:5d}".format(int(res[3])))
if(res[3] == max_iter):
    print("    >> unstable or incomplete relaxation")
print("  average stability of magnetization at last iteration: {0:.4e} T".format(res[0]))
print("  maximum absolute magnetization at last iteration: {0:.5f} T".format(res[1]))
print("  maximum H vector at last iteration: {0:.5f} T".format(res[2]))
# print("Pole-tip magnetic field: {0:.8f} T".format(rad.Fld(quad, 'bz', [x,y,z])))b

The quadrupole gradient ... $B_z$ field at the origin, $(0, 0, 0)$, is the vertical
field in the middle of the gap.

In [ ]:
Bz  = rad.Fld(quad, 'Bz', [0, 1, 0]) * 1e3
Bz1 = rad.Fld(quad, 'Bz', [0,10, 0]) * 1e3 / 10
Iz  = rad.FldInt(quad, 'inf', 'ibz', [-1, 1, 0], [1, 1, 0])
Iz1 = rad.FldInt(quad, 'inf', 'ibz', [-1,10, 0], [1,10, 0]) / 10

print(' quadrupole gradient:', round(Bz, 4), 'T/m')
print('  int.quad. at  1 mm:', round(Iz, 5), 'T')
print('δ int.quad. at 10 mm:', round((Iz1 / Iz - 1), 6) * 100, '%') # rel. var. in field integral

## _Plots of the magnetic field and field integrals_

Here we show plots of magnetic field in the gap and corresponding
field integrals. The field values are obtained by calling `Fld` on
a list of points. One may also use `FldLst`.

The first graph here displays ...

In [ ]:
# plots of magnetic field
z    = 0
x1   = 0
x2   = 30
ymax = 40
np   = 20

BzVy1 = rad.FldLst(quad, 'bz', [x1, 0, z], [x1, ymax, z], np, 'arg', 0)
BzVy2 = rad.FldLst(quad, 'bz', [x2, 0, z], [x2, ymax, z], np, 'arg', 0)

uti_plot1d_m([BzVy1, BzVy2],
             labels = ['Y', 'Vertical Magnetic Field', 'Vertical Magnetic Field vs. Horizontal Position'], units = ['mm', 'T'],
             styles = ['-b.', '-r.'], legend = ['X = {} mm'.format(x1), 'X = {} mm'.format(x2)])

xmax = 1.5 * thick
y1 = width / 4
y2 = width / 2

BzVx1 = rad.FldLst(quad, 'bz', [-xmax, y1, 0], [xmax, y1, 0], 2 * np, 'arg', 0)
BzVx2 = rad.FldLst(quad, 'bz', [-xmax, y2, 0], [xmax, y2, 0], 2 * np, 'arg', 0)

uti_plot1d_m([BzVx1, BzVx2],
             labels = ['X', 'Vertical Magnetic Field', 'Vertical Magnetic Field vs. Longitudinal Position'], units = ['mm', 'T'],
             styles = ['-b.', '-r.'], legend = ['Y = {} mm'.format(y1), 'Y = {} mm'.format(y2)])

# plots of magnetic field integrals
z    = 0
ymin = 0.001
ymax = 10
npy  = 20
yStep = (ymax - ymin) / (npy - 1)
IBz0 = rad.FldInt(quad, 'inf', 'ibz', [-1, 1, z], [1, 1, z])

IBzVsY = [(rad.FldInt(quad, 'inf', 'ibz', [-1, ymin + iy * yStep, z], [1,ymin+iy*yStep,z]) /
           ((ymin+iy*yStep)*IBz0) - 1) * 100 for iy in range(npy)]
uti_plot1d(IBzVsY, [ymin, ymax, npy],
           ['Y', 'dIz', 'Rel. Variation of Vertical Field Integral along X at Z = ' + repr(z) + ' mm'], ['mm', '%'])

# harmonic analysis of the field integrals
nharm = 10; radius = 2; y = 0; z = 0;
w = harm(quad, y, z, radius, nharm);

mm = [multipole(w, i) for i in range(nharm)];
round_mm = [complex(round(mm[i].real, 9), round(mm[i].imag, 9)) for i in range(nharm)];
print(round_mm)

uti_plot_show()

### Compare results

In [ ]:
BzVy1_sv = numpy.loadtxt(nb_dir + "BzVy1.txt")
BzVy2_sv = numpy.loadtxt(nb_dir + "BzVy2.txt")
BzVx1_sv = numpy.loadtxt(nb_dir + "BzVx1.txt")
BzVx2_sv = numpy.loadtxt(nb_dir + "BzVx2.txt")

In [ ]:
uti_plot1d_m([BzVy1, BzVy2, BzVy1_sv, BzVy2_sv],
             labels = ['Y', 'Vertical Magnetic Field', 'Vertical Magnetic Field vs. Horizontal Position'], units = ['mm', 'T'],
             styles = ['-b.', '-r.', '--g.', '--g.'], legend = ['X = {} mm'.format(x1), 'X = {} mm'.format(x2)])

uti_plot1d_m([BzVx1, BzVx2, BzVx1_sv, BzVx2_sv],
             labels = ['X', 'Vertical Magnetic Field', 'Vertical Magnetic Field vs. Longitudinal Position'], units = ['mm', 'T'],
             styles = ['-b.', '-r.', '--g.', '--g.'], legend = ['Y = {} mm'.format(y1), 'Y = {} mm'.format(y2)])

uti_plot_show()